## R script for OMOP to load EPA Air Quality data

### setup

In [1]:
install.packages("config")
install.packages("DBI")
library(config)
library(DBI)

# make db connection
# for format of database file see db/env/db_conf.txt
db <- read.delim( '../../db/env/feta.txt', header=TRUE, sep=' ' )
con <- dbConnect(RPostgres::Postgres(),
                 dbname = toString(db$database[1]),
                 host = toString(db$host[1]),
                 port = 5432,
                 user = toString(db$user),
                 password = toString(db$pass))

# check the connection
dbListTables(con)

[1] "geography_columns"               "geometry_columns"               
 [3] "spatial_ref_sys"                 "acs_income_bmi"                 
 [5] "attr_epa_aqi_2019"               "attr_florida_acs"               
 [7] "data_source"                     "care_site"                      
 [9] "condition_occurrence"            "attr_dindex"                    
[11] "attr_florida_tri_2018"           "dindex_bmi"                     
[13] "geo_florida_tri_2018"            "geo_miamidade_census_tract_2018"
[15] "measurement"                     "md_hexbins_1km"                 
[17] "geo_us_counties"                 "locations"                      
[19] "utm_grid"                        "persons"                        
[21] "visit_occurrence"                "sites"                          
[23] "attr_index"                      "geo_index"                      
[25] "site_history"                    "location_history"               
[27] "geo_point_template"              "attr_template"                  
[29] "geo_fl_epa_aqs_sites"            "attr_fl_epa_pm25_2018"

### get EPA data from API  
note that we use annual averages

In [1]:
#install.packages("httr")
library(httr)
base_api_url <- "https://aqs.epa.gov/data/api/annualData/"
api_query_type <- "byState"
api_email <- "test@aqs.api"
api_key <- "test"
api_param <- "88101,88502"  # PM2.5 88101,88502; NO2 42602
                            # https://aqs.epa.gov/aqsweb/documents/codetables/methods_all.html
api_bdate <- "20180115"     # begin date
api_edate <- "20180115"     # end date: set to same as bdate for whole year
api_state <- "12"           # Florida

request_url <- paste(
    base_api_url,api_query_type,"?", 
    "email=",api_email, 
    "&key=",api_key,
    "&param=",api_param,
    "&bdate=",api_bdate,
    "&edate=",api_bdate,
    "&state=",api_state,
    sep=""
)
res <- GET(request_url)
# Todo: error checking on response
data <- content(res)$Data

### get EPA data from file

In [ ]:
base_download_url <- "https://aqs.epa.gov/aqsweb/airdata/"
file_download <- "annual_conc_by_monitor_2020.zip"

# all us sites:
file_download <- "aqs_sites.zip"

# all us monitors
file_download <- "aqs_monitors.zip"

In [2]:
# all sites (i.e. monitor locations)
base_download_url <- "https://aqs.epa.gov/aqsweb/airdata/"
file_download <- "aqs_sites.zip"
file_name <- "aqs_sites.csv"

temp <- tempfile()
download.file(paste(base_download_url,file_download,sep=""),temp)
data <- read.csv(unz(temp, file_name), header=TRUE)
unlink(temp)

In [2]:
data

[[1]]
[[1]]$state_code
[1] "12"

[[1]]$county_code
[1] "057"

[[1]]$site_number
[1] "3002"

[[1]]$parameter_code
[1] "88101"

[[1]]$poc
[1] 3

[[1]]$latitude
[1] 27.96565

[[1]]$longitude
[1] -82.2304

[[1]]$datum
[1] "WGS84"

[[1]]$parameter
[1] "PM2.5 - Local Conditions"

[[1]]$sample_duration
[1] "24-HR BLK AVG"

[[1]]$pollutant_standard
[1] "PM25 24-hour 2006"

[[1]]$metric_used
[1] "Daily Mean"

[[1]]$method
NULL

[[1]]$year
[1] 2018

[[1]]$units_of_measure
[1] "Micrograms/cubic meter (LC)"

[[1]]$event_type
[1] "No Events"

[[1]]$observation_count
[1] 300

[[1]]$observation_percent
[1] 82

[[1]]$validity_indicator
[1] "N"

[[1]]$valid_day_count
[1] 300

[[1]]$required_day_count
[1] 365

[[1]]$exceptional_data_count
[1] 0

[[1]]$null_observation_count
[1] 0

[[1]]$primary_exceedance_count
[1] 0

[[1]]$secondary_exceedance_count
[1] 0

[[1]]$certification_indicator
[1] "Requested but denied"

[[1]]$arithmetic_mean
[1] 7.944667

[[1]]$standard_deviation
[1] 3.885416

[[1]]$first_max_value
[1] 24.8

[[1]]$first_max_datetime
[1] "2018-01-01 00:00"

[[1]]$second_max_value
[1] 22.1

[[1]]$second_max_datetime
[1] "2018-03-16 00:00"

[[1]]$third_max_value
[1] 22

[[1]]$third_max_datetime
[1] "2018-01-19 00:00"

[[1]]$fourth_max_value
[1] 21.2

[[1]]$fourth_max_datetime
[1] "2018-01-18 00:00"

[[1]]$first_max_nonoverlap_value
NULL

[[1]]$first_max_n_o_datetime
NULL

[[1]]$second_max_nonoverlap_value
NULL

[[1]]$second_max_n_o_datetime
NULL

[[1]]$ninety_ninth_percentile
[1] 22

[[1]]$ninety_eighth_percentile
[1] 17.3

[[1]]$ninety_fifth_percentile
[1] 15

[[1]]$ninetieth_percentile
[1] 13.4

[[1]]$seventy_fifth_percentile
[1] 9.9

[[1]]$fiftieth_percentile
[1] 7.2

[[1]]$tenth_percentile
[1] 4

[[1]]$local_site_name
[1] "SYDNEY"

[[1]]$site_address
[1] "1167 N Dover Road, Valrico, FL 33527"

[[1]]$state
[1] "Florida"

[[1]]$county
[1] "Hillsborough"

[[1]]$city
[1] "Valrico"

[[1]]$cbsa_code
[1] "45300"

[[1]]$cbsa
[1] "Tampa-St. Petersburg-Clearwater, FL"

[[1]]$date_of_last_change
[1] "2020-05-21"


[[2]]
[[2]]$state_code
[1] "12"

[[2]]$county_code
[1] "057"

[[2]]$site_number
[1] "3002"

[[2]]$parameter_code
[1] "88101"

[[2]]$poc
[1] 2

[[2]]$latitude
[1] 27.96565

[[2]]$longitude
[1] -82.2304

[[2]]$datum
[1] "WGS84"

[[2]]$parameter
[1] "PM2.5 - Local Conditions"

[[2]]$sample_duration
[1] "24 HOUR"

[[2]]$pollutant_standard
[1] "PM25 24-hour 2006"

[[2]]$metric_used
[1] "Daily Mean"

[[2]]$method
[1] "R & P Model 2025 PM-2.5 Sequential Air Sampler w/VSCC - Gravimetric"

[[2]]$year
[1] 2018

[[2]]$units_of_measure
[1] "Micrograms/cubic meter (LC)"

[[2]]$event_type
[1] "No Events"

[[2]]$observation_count
[1] 21

[[2]]$observation_percent
[1] 68

[[2]]$validity_indicator
[1] "N"

[[2]]$valid_day_count
[1] 21

[[2]]$required_day_count
[1] 31

[[2]]$exceptional_data_count
[1] 0

[[2]]$null_observation_count
[1] 14

[[2]]$primary_exceedance_count
[1] 0

[[2]]$secondary_exceedance_count
[1] 0

[[2]]$certification_indicator
[1] "Certified"

[[2]]$arithmetic_mean
[1] 6.814286

[[2]]$standard_deviation
[1] 2.675123

[[2]]$first_max_value
[1] 15.1

[[2]]$first_max_datetime
[1] "2018-07-22 00:00"

[[2]]$second_max_value
[1] 11.3

[[2]]$second_max_datetime
[1] "2018-04-20 00:00"

[[2]]$third_max_value
[1] 10.2

[[2]]$third_max_datetime
[1] "2018-03-15 00:00"

[[2]]$fourth_max_value
[1] 8.7

[[2]]$fourth_max_datetime
[1] "2018-12-04 00:00"

[[2]]$first_max_nonoverlap_value
NULL

[[2]]$first_max_n_o_datetime
NULL

[[2]]$second_max_nonoverlap_value
NULL

[[2]]$second_max_n_o_datetime
NULL

[[2]]$ninety_ninth_percentile
[1] 15.1

[[2]]$ninety_eighth_percentile
[1] 15.1

[[2]]$ninety_fifth_percentile
[1] 11.3

[[2]]$ninetieth_percentile
[1] 10.2

[[2]]$seventy_fifth_percentile
[1] 7.7

[[2]]$fiftieth_percentile
[1] 6

[[2]]$tenth_percentile
[1] 4.9

[[2]]$local_site_name
[1] "SYDNEY"

[[2]]$site_address
[1] "1167 N Dover Road, Valrico, FL 33527"

[[2]]$state
[1] "Florida"

[[2]]$county
[1] "Hillsborough"

[[2]]$city
[1] "Valrico"

[[2]]$c

### create empty tables in database

- TODO: define table naming conventions
- TODO: check for exisiting tables
- TODO: update geo_index and attr_index tables with metadata
- TODO: index geospatial columns for faster query
- NOTE: using empty template tables, not specification

In [11]:
# create empty tables for geometry and attributes

geo_tablename <- 'geo_fl_epa_aqs_sites'
sql_createidseq <- paste("CREATE SEQUENCE ",geo_tablename,"_id_seq ", # geo table id sequence
                         "INCREMENT 1 START 1 ",
                         "MINVALUE 1 MAXVALUE 9223372036854775807 ",
                         "CACHE 1;",
                         sep="")
req <- dbSendQuery(con,sql_createidseq)
dbClearResult(req)

sql_createpointgeom <- paste("CREATE TABLE",geo_tablename, # geo table as clone
                             "as SELECT * FROM geo_point_template",
                             "WITH NO DATA;")
req <- dbSendQuery(con,sql_createpointgeom)
dbClearResult(req)

sql_alterpointgeom <- paste("ALTER TABLE ",geo_tablename, # geo table add keys and relations
                            " ADD CONSTRAINT ",geo_tablename,"_id_pkey ",
                            "PRIMARY KEY (geo_record_id), ",
                            "ALTER COLUMN geo_record_id ",
                            "SET DEFAULT nextval('",geo_tablename,"_id_seq","'::regclass);",
                            sep="")
req <- dbSendQuery(con,sql_alterpointgeom)
dbClearResult(req)

att_tablename <- 'attr_fl_epa_pm25_2018'
sql_createidseq <- paste("CREATE SEQUENCE ",att_tablename,"_id_seq ", # attr table id seq
                         "INCREMENT 1 START 1 ",
                         "MINVALUE 1 MAXVALUE 9223372036854775807 ",
                         "CACHE 1;",
                         sep="")
req <- dbSendQuery(con,sql_createidseq)
dbClearResult(req)

sql_createattr <- paste("CREATE TABLE",att_tablename, # attr table as clone
                        "AS SELECT * FROM attr_template",
                        "WITH NO DATA;")
req <- dbSendQuery(con,sql_createattr)
dbClearResult(req)

sql_alterattr <- paste("ALTER TABLE ",att_tablename, #attr table add keys and relations
                       " ADD CONSTRAINT ",att_tablename,"_id_pkey ",
                       "PRIMARY KEY (attr_record_id), ",
                       "ALTER COLUMN attr_record_id ",
                       "SET DEFAULT nextval('",att_tablename,"_id_seq","'::regclass), ",
                       "ADD CONSTRAINT ",att_tablename,"_geo_record_id_fkey ",
                       "FOREIGN KEY (geo_record_id) ",
                       "REFERENCES ",geo_tablename," (geo_record_id) MATCH SIMPLE ",
                       "ON UPDATE NO ACTION ON DELETE NO ACTION;",
                       sep="")
req <- dbSendQuery(con,sql_alterattr)
dbClearResult(req)

### load geospatial site data

In [12]:
sites <- list()
for (site in data){
    
    if (! site$site_number %in% sites) {

        # create location with geometry
        sites <- c(site$site_number,sites)
        sql_createsite <- paste(
            'INSERT INTO',geo_tablename,' ("geo_record_id","name","geom_wgs84") 
            VALUES (DEFAULT,$3,ST_SetSRID(ST_MakePoint($2,$1),4326))
            RETURNING geo_record_id'
        )
        res <- dbSendQuery(con,sql_createsite)
        dbBind(res, list(site$lat,site$lon,paste(site$site_number,":",site$local_site_name)))
        site_ID <- dbFetch(res)[1,1]
        dbClearResult(res)
    
    }

}

In [16]:
dbReadTable(con, geo_tablename) 

geo_record_id,name,source_id_coding,source_id_value,geom_wgs84,geom_local
<int>,<chr>,<chr>,<chr>,<pq_gmtry>,<pq_gmtry>
1,3002 : SYDNEY,NA,NA,0101000020E6100000E63FA4DFBE8E54C0E561A1D634F73B40,NA
2,0013 : Bee Ridge Park,NA,NA,0101000020E6100000F373435376A054C015A8C5E0614A3B40,NA
3,2003 : Pompano Highlands,NA,NA,0101000020E6100000C6F7C5A52A0654C00F4240BE844A3A40,NA
4,0033 : Palm Springs Fire Station,NA,NA,0101000020E61000008C31B08EE31454C0D6A9F23D23F13940,NA
5,0004 : Ellyson Industrial Park,NA,NA,0101000020E6100000744694F606CD55C0CB65A3737E863E40,NA
6,5005 : Coconut Creek,NA,NA,0101000020E6100000F29716F5490B54C0FC8F4C874E4B3A40,NA
7,0012 : Tallahassee Community College,NA,NA,0101000020E61000006DACC43C2B1655C0EB54F99E91703E40,NA
8,1002 : Sanford (Seminole Community College),NA,NA,0101000020E6100000E7E44526E05354C0942F682101BF3C40,NA
9,0018 : Azalea Park,NA,NA,0101000020E61000003108AC1C5AAF54C0D009A1832EC93B40,NA


In [8]:
n <- 0
for (site in data) {
    #print (paste(site$site_number,site$local_site_name))
    if (site$site_number=='5002') {
        print (
            paste(
                #site$site_number,
                #site$parameter_code,
                site$arithmetic_mean,'|',
                #site$date_of_last_change,
                #site$parameter,
                #site$units_of_measure,
                site$observation_count,'|',
                #site$observation_percent,
                site$valid_day_count,'|',
                site$method,'|',
                site$pollutant_standard,'|',
                site$first_max_datetime,'|',
                site$first_max_value
            )
        )
        print ('____________________________________')
        n <- n + 1
    }
}
print(n)

[1] "7.233989 | 356 | 356 | Multiple Methods Used | PM25 Annual 2012 | 2018-01-20 00:00 | 23.6"
[1] "____________________________________"
[1] "7.233989 | 356 | 356 | Multiple Methods Used | PM25 24-hour 2012 | 2018-01-20 00:00 | 23.6"
[1] "____________________________________"
[1] "7.233989 | 356 | 356 | Multiple Methods Used | PM25 Annual 2006 | 2018-01-20 00:00 | 23.6"
[1] "____________________________________"
[1] "7.233989 | 356 | 356 | Multiple Methods Used | PM25 24-hour 2006 | 2018-01-20 00:00 | 23.6"
[1] "____________________________________"
[1] "7.270524 | 8573 | 364 | Multiple Methods Used |  | 2018-01-01 00:00 | 59.7"
[1] "____________________________________"
[1] "6.581034 | 58 | 58 | R & P Model 2025 PM-2.5 Sequential Air Sampler w/VSCC - Gravimetric | PM25 Annual 2012 | 2018-01-20 00:00 | 20.2"
[1] "____________________________________"
[1] "6.581034 | 58 | 58 | R & P Model 2025 PM-2.5 Sequential Air Sampler w/VSCC - Gravimetric | PM25 24-hour 2012 | 2018-01-20 00:00 | 

In [25]:
cols = names(site)

In [61]:
for(key in names(site)){
  value<-site[key]
  print(paste(key,'=',value))
}

[1] "state_code = 12"
[1] "county_code = 017"
[1] "site_number = 0006"
[1] "parameter_code = 88502"
[1] "poc = 1"
[1] "latitude = 28.958644"
[1] "longitude = -82.642965"
[1] "datum = WGS84"
[1] "parameter = Acceptable PM2.5 AQI & Speciation Mass"
[1] "sample_duration = 1 HOUR"
[1] "pollutant_standard = NULL"
[1] "metric_used = Observed Values"
[1] "method = PM2.5 SCC w/Correction Factor - TEOM Gravimetric 50 deg C"
[1] "year = 2018"
[1] "units_of_measure = Micrograms/cubic meter (LC)"
[1] "event_type = No Events"
[1] "observation_count = 8270"
[1] "observation_percent = 94"
[1] "validity_indicator = Y"
[1] "valid_day_count = 344"
[1] "required_day_count = 365"
[1] "exceptional_data_count = 0"
[1] "null_observation_count = 490"
[1] "primary_exceedance_count = NULL"
[1] "secondary_exceedance_count = NULL"
[1] "certification_indicator = Certification not required"
[1] "arithmetic_mean = 8.394353"
[1] "standard_deviation = 4.325292"
[1] "first_max_value = 63"
[1] "first_max_datetime = 2018

In [94]:
which(data[[]]$site_number=='0035')

integer(0)

In [97]:
lapply(
    data, function(x) 
        filter (x, site_number=='0035')
)

ERROR: Error in storage.mode(x) <- "double": 'list' object cannot be coerced to type 'double'


In [113]:
filter(data, site_number=="0035")

ERROR: Error in storage.mode(x) <- "double": 'list' object cannot be coerced to type 'double'


In [104]:
site

$state_code
[1] "12"

$county_code
[1] "017"

$site_number
[1] "0006"

$parameter_code
[1] "88502"

$poc
[1] 1

$latitude
[1] 28.95864

$longitude
[1] -82.64297

$datum
[1] "WGS84"

$parameter
[1] "Acceptable PM2.5 AQI & Speciation Mass"

$sample_duration
[1] "1 HOUR"

$pollutant_standard
NULL

$metric_used
[1] "Observed Values"

$method
[1] "PM2.5 SCC w/Correction Factor - TEOM Gravimetric 50 deg C"

$year
[1] 2018

$units_of_measure
[1] "Micrograms/cubic meter (LC)"

$event_type
[1] "No Events"

$observation_count
[1] 8270

$observation_percent
[1] 94

$validity_indicator
[1] "Y"

$valid_day_count
[1] 344

$required_day_count
[1] 365

$exceptional_data_count
[1] 0

$null_observation_count
[1] 490

$primary_exceedance_count
NULL

$secondary_exceedance_count
NULL

$certification_indicator
[1] "Certification not required"

$arithmetic_mean
[1] 8.394353

$standard_deviation
[1] 4.325292

$first_max_value
[1] 63

$first_max_datetime
[1] "2018-04-06 20:00"

$second_max_value
[1] 51.3

$second_max_datetime
[1] "2018-04-06 19:00"

$third_max_value
[1] 45

$third_max_datetime
[1] "2018-03-14 19:00"

$fourth_max_value
[1] 37.7

$fourth_max_datetime
[1] "2018-03-15 11:00"

$first_max_nonoverlap_value
NULL

$first_max_n_o_datetime
NULL

$second_max_nonoverlap_value
NULL

$second_max_n_o_datetime
NULL

$ninety_ninth_percentile
[1] 23.1

$ninety_eighth_percentile
[1] 20.3

$ninety_fifth_percentile
[1] 16.4

$ninetieth_percentile
[1] 13.5

$seventy_fifth_percentile
[1] 10.2

$fiftieth_percentile
[1] 7.7

$tenth_percentile
[1] 3.9

$local_site_name
[1] "Crystal River Preserve"

$site_address
[1] "13450 W. Power Line Rd., Crystal River, FL 34428"

$state
[1] "Florida"

$county
[1] "Citrus"

$city
NULL

$cbsa_code
[1] "26140"

$cbsa
[1] "Homosassa Springs, FL"

$date_of_last_change
[1] "2020-05-21"